In [1]:
%matplotlib inline
import numpy as np
import math as mth
from numpy import polynomial as pn

** Три параллельных графитовых электрода дуговой печи переменного тока имеют длину $ 3 м $ каждый. Расстояние между ними равно $ b = 0.9 м $. Определить среднюю электромагнитную силу, действующую на каждый электрод при амплитудном значении тока $ I_{0} = 10 кА $. Сдвиг по фазе между текущими электродами принять $ \phi = 120^{\circ} $.**

***На единицу длины тока действует мгновенная сила*** $$ F = \frac{\mu_{0} I_{1} I_{2}}{2 \pi b}, \mu_{0} = 4\pi\bullet10^{-7} Г/м. $$

In [2]:
def trapezoidal_rule(function, a, b, n):
    result = 0
    h = (b - a) / n
    for x in np.arange(a + h, b, h):
        result += function(x)
    result = ((function(a) + function(b)) / 2 + result) * h
    return result

def simpsons_rule(function, a, b, n):
    result = 0
    h = (b - a) / n
    for x in range(1, n):
        if x % 2 == 0:
            result += 2 * function(a + x*h)
        else:
            result += 4 * function(a + x*h)
    result = h*(result + function(a) + function(b)) / 3
    return result

def gauss_rule(function, a, b, n):
    def calculate(n):
        result = 0.0
        t_list, a_list = pn.legendre.leggauss(n)
        for a_i, t_i in zip(a_list, t_list):
            result += a_i * function((b + a) / 2 + t_i * (b - a) / 2)
        result *= (b - a)/2
        return result
    return calculate(n)
        
def runge_rule(function, a, b, eps):
    def calculate(rule, k):
        n = 10
        delta = 1
        result_n = 0.0
        while delta > eps:
            result_n = rule(function, a, b, n)
            result_2n = rule(function, a, b, 2* n)
            n *= 2
            delta = abs(result_n - result_2n)/k
        return result_n
    return (calculate(trapezoidal_rule, 3), 
            calculate(simpsons_rule, 15))

In [3]:
I_0 = 10e3 #
phi = 4 * mth.pi/3 #
T = 1/50 #
omega = 2 * mth.pi/T #
l = 3 #
b = 0.9 #
mu_0 = 4e-7 * mth.pi #
k = l * mu_0/(2 * b * T * mth.pi) #
epsilon = 1e-6

i_1 = lambda t : I_0 * mth.sin(omega * t)

i_2 = lambda t : I_0 * mth.sin(omega * t + phi)

i_3 = lambda t : I_0 * mth.sin(omega * t - phi)

**Вычисление интегралов: $$ F_{12} = \frac{\mu_{0} l}{2\pi T} \int_{0}^{T} i_1(t)i_2(t) dt $$ **

In [4]:
int_f_12 = lambda t : 25e4 * (-100 * t - mth.sin(200 * mth.pi + 2 * mth.pi/3)/mth.pi)

f_12 = list(runge_rule(lambda x : i_1(x) * i_2(x), epsilon, T, 1e-6))
f_12.append(gauss_rule(lambda x : i_1(x) * i_2(x), epsilon, T, 61))
f_12.append(int_f_12(T) - int_f_12(0))

print('Вычисление методом трапеций:',k * f_12[0])
print('Вычисление методом Симпсона:',k * f_12[1])
print('Вычисление методом Гаусса:', k * f_12[2])
print('Истинное значение:', k * f_12[3])

Вычисление методом трапеций: -16.666666213057454
Вычисление методом Симпсона: -16.666666213338182
Вычисление методом Гаусса: -16.6666662132
Истинное значение: -16.66666666666666


** $$ F_{23} = \frac{\mu_{0} l}{2\pi T} \int_{0}^{T} i_2(t)i_3(t) dt $$ **

In [5]:
int_f_23 = lambda t : 25e4 * (-100 * t - mth.sin(200 * mth.pi)/mth.pi)

f_23 = list(runge_rule(lambda x : i_2(x) * i_3(x), epsilon, T, 1e-6))
f_23.append(gauss_rule(lambda x : i_2(x) * i_3(x), epsilon, T, 61))
f_23.append(int_f_23(T))

print('Вычисление методом трапеций:',k * f_23[0])
print('Вычисление методом Симпсона:',k * f_23[1])
print('Вычисление методом Гаусса:', k * f_23[2])
print('Истинное значение:', k * f_23[3])

Вычисление методом трапеций: -16.66416666682876
Вычисление методом Симпсона: -16.664166666423803
Вычисление методом Гаусса: -16.6641666668
Истинное значение: -16.666666666666675


** $$ F_{13} = \frac{\mu_{0} l}{2\pi T} \int_{0}^{T} i_1(t)i_3(t) dt $$ **

In [6]:
int_f_13 = lambda t : 25e4 * (-100 * t - mth.sin(200 * mth.pi - 2*mth.pi/3)/mth.pi)

f_13 = list(runge_rule(lambda x : i_1(x) * i_3(x), epsilon, T, 1e-6))
f_13.append(gauss_rule(lambda x : i_1(x) * i_3(x), epsilon, T, 61))
f_13.append(int_f_13(T) - int_f_13(0))

print('Вычисление методом трапеций:',k * f_13[0])
print('Вычисление методом Симпсона:',k * f_13[1])
print('Вычисление методом Гаусса:', k * f_13[2])
print('Истинное значение:', k * f_13[3])

Вычисление методом трапеций: -16.666667119957026
Вычисление методом Симпсона: -16.666667120238017
Вычисление методом Гаусса: -16.6666671201
Истинное значение: -16.666666666666664


** Вычисление общей силы:$$ F_{1рез} = \sqrt{F_{12}^2 + F_{13}^2 - 2 F_{12} F_{13} cos 120^\circ} $$
$$ F_{2рез} = \sqrt{F_{12}^2 + F_{23}^2 - 2 F_{12} F_{23} cos 120^\circ} $$
$$ F_{3рез} = \sqrt{F_{23}^2 + F_{13}^2 - 2 F_{23} F_{13} cos 120^\circ} $$**

In [7]:
print('Результат, полученный решением методом трапеций:')
f_1_res = k * mth.sqrt(f_12[0] ** 2 + f_13[0] ** 2 - 2 * f_12[0] * f_13[0] * mth.cos(phi))
print('F1 =',f_1_res)

f_2_res = k * mth.sqrt(f_12[0] ** 2 + f_23[0] ** 2 - 2 * f_12[0] * f_23[0] * mth.cos(phi))
print('F2 =',f_2_res)

f_3_res = k * mth.sqrt(f_23[0] ** 2 + f_13[0] ** 2 - 2 * f_23[0] * f_13[0] * mth.cos(phi))
print('F3 =',f_3_res)


print('\nРезультат, полученный решением методом Симпсона:')
f_1_res = k * mth.sqrt(f_12[1] ** 2 + f_13[1] ** 2 - 2 * f_12[1] * f_13[1] * mth.cos(phi))
print('F1 =',f_1_res)

f_2_res = k * mth.sqrt(f_12[1] ** 2 + f_23[1] ** 2 - 2 * f_12[1] * f_23[1] * mth.cos(phi))
print('F2 =',f_2_res)

f_3_res = k * mth.sqrt(f_23[1] ** 2 + f_13[1] ** 2 - 2 * f_23[1] * f_13[1] * mth.cos(phi))
print('F3 =',f_3_res)


print('\nРезультат, полученный решением методом Гаусса:')
f_1_res = k * mth.sqrt(f_12[2] ** 2 + f_13[2] ** 2 - 2 * f_12[2] * f_13[2] * mth.cos(phi))
print('F1 =',f_1_res)

f_2_res = k * mth.sqrt(f_12[2] ** 2 + f_23[2] ** 2 - 2 * f_12[2] * f_23[2] * mth.cos(phi))
print('F2 =',f_2_res)

f_3_res = k * mth.sqrt(f_23[2] ** 2 + f_13[2] ** 2 - 2 * f_23[2] * f_13[2] * mth.cos(phi))
print('F3 =',f_3_res)


print('\nРезультат, полученный решением первообразной:')
f_1_res = k * mth.sqrt(f_12[3] ** 2 + f_13[3] ** 2 - 2 * f_12[3] * f_13[3] * mth.cos(phi))
print('F1 =',f_1_res)

f_2_res = k * mth.sqrt(f_12[3] ** 2 + f_23[3] ** 2 - 2 * f_12[3] * f_23[3] * mth.cos(phi))
print('F2 =',f_2_res)

f_3_res = k * mth.sqrt(f_23[3] ** 2 + f_13[3] ** 2 - 2 * f_23[3] * f_13[3] * mth.cos(phi))
print('F3 =',f_3_res)

Результат, полученный решением методом трапеций:
F1 = 28.86751345920516
F2 = 28.865348030330605
F3 = 28.865348815748312

Результат, полученный решением методом Симпсона:
F1 = 28.867513459691622
F2 = 28.865348030223036
F3 = 28.86534881564097

Результат, полученный решением методом Гаусса:
F1 = 28.867513459386334
F2 = 28.865348030375735
F3 = 28.865348815793503

Результат, полученный решением первообразной:
F1 = 28.867513459481287
F2 = 28.867513459481295
F3 = 28.867513459481298
